In [2]:
import requests
import pandas as pd
import json
from scripts.env import HEADERS
from scripts.fravega import get_product_data, get_product_attributes
import os

In [3]:
max_products = 10
productos = os.listdir('datos/asus/productos')

In [18]:
dataframes = []
for p in productos:
    with open(f'datos/asus/productos/{p}', 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    dataframes.append(pd.DataFrame([get_product_attributes(data)]))

In [21]:
df_atributos = pd.concat(dataframes)

In [23]:
df_productos = pd.read_csv('datos/asus/productos-asus-fravega.csv')
df_productos.head()

,id,title,katalogCategoryId,slug,brand_id,brand_name,sku_id,categories,list_price,sale_price,product_url
0,68f0f5deea8fd27339dce13e,"ASUS C523NA Chromebook 15.6 HD portátil,",5a301c031400008a0049181b,asus-c523na-chromebook-15-6-hd-portatil-,5a301bdc140000630049072b,Asus,22629527,"['Notebooks', 'Informática']",461318.0,392121.0,https://www.fravega.com/p/asus-c523na-chromebo...
1,68f0f5ee5ed3d8a15f2f4e5d,ASUS CX1500CNA Chromebook 15.6andquot; HD,5a301c031400008a0049181b,asus-cx1500cna-chromebook-15-6andquot-hd,5a301bdc140000630049072b,Asus,22629529,"['Notebooks', 'Informática']",435435.0,370120.0,https://www.fravega.com/p/asus-cx1500cna-chrom...
2,68f04026e6caf9935203e207,ASUS Chromebook CM1402 - Portátil antirreflejos,5a301c031400008a0049181b,asus-chromebook-cm1402-portatil-antirreflejos,5a301bdc140000630049072b,Asus,22629114,"['Notebooks', 'Informática']",1088588.0,925300.0,https://www.fravega.com/p/asus-chromebook-cm14...
3,68f10442ddac6589db84c69f,ASUS Chromebook CM1402 - Portátil antirreflejos,5a301c031400008a0049181b,asus-chromebook-cm1402-portatil-antirreflejos,5a301bdc140000630049072b,Asus,22630095,"['Notebooks', 'Informática']",645540.0,548709.0,https://www.fravega.com/p/asus-chromebook-cm14...
4,68f10870132e21ca4580e396,ASUS Chromebook CM1402 Portátil FHD de 14,5a301c031400008a0049181b,asus-chromebook-cm1402-portatil-fhd-de-14,5a301bdc140000630049072b,Asus,22630238,"['Notebooks', 'Informática']",784088.0,666475.0,https://www.fravega.com/p/asus-chromebook-cm14...


In [25]:
df_atributos.head()

,product_id,product_specifications,product_attributes
0,6138918d100000370000f3dc,[{'Modelo': '1'}],"[{'Compra Internacional': False}, {'Modelo': '..."
0,64c3e562340000d9a7e0eb21,[{'Origen': 'Argentina'}],"[{'Compra Internacional': False}, {'Origen': '..."
0,66fed902250000182ca2a952,"[{'Modelo': 'RTX3050'}, {'Origen': 'China'}, {...","[{'Compra Internacional': False}, {'Modelo': '..."
0,6704b8f5400000b808f37464,[{'Origen': 'Argentina'}],"[{'Compra Internacional': False}, {'Origen': '..."
0,670fd0bf400000e29de0353e,"[{'Procesador': 'Celeron'}, {'Disco SSD': '128...","[{'Notebook Gamer': False}, {'Tipo de procesad..."


In [28]:
full = pd.merge(

    df_productos,
    df_atributos,
    left_on='id',
    right_on='product_id',
    how='outer'
)


In [33]:
full = full.dropna().sort_values('title').reset_index(drop=True).drop(columns=['product_id'])

In [35]:
full.to_csv('datos/asus/productos-asus.csv', index=False)